In [11]:
import tensorflow as tf
import pickle
import numpy as np

import foolbox
import eagerpy as ep
from tqdm.notebook import tqdm

In [7]:
batch_size= 16
test_batches = tf.keras.utils.image_dataset_from_directory(
    "../dr/diabetic-retinopathy-2015-data-colored-resized/colored_images",
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(224, 224),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset="validation"
)
test_batches = test_batches.map(lambda x,y: (tf.keras.applications.resnet50.preprocess_input(x),y))

Found 35126 files belonging to 2 classes.
Using 7025 files for validation.


In [8]:
# with open("../dr/DR_model.pkl","rb") as file:
#     model_without_attention = pickle.load(file)
with open("../dr/DR+SA_model.pkl","rb") as file:
    model_with_attention = pickle.load(file)

In [18]:
batch_accs = []
batch_weights = []
for i,(x,y) in zip(range(len(test_batches)),test_batches):
    # if i == 0:
    #     print(type(x),y)
    acc = fb.utils.accuracy(fmodel, x, tf.argmax(y, axis=1))
    batch_accs.append(acc)
    batch_weights.append(y.shape[0])
s1 = 0
s2 = sum(batch_weights)
for i in range(len(batch_accs)):
    s1 += batch_accs[i] * batch_weights[i]
print("unperturbed acc:",s1/s2)


unperturbed acc: 0.817508896797153


In [9]:
model = model_with_attention
preprocessing = dict()
bounds = (-123.69, 151.062)
# preprocessing = dict(flip_axis=-1, mean=[103.939, 116.779, 123.68])
# bounds = (0, 255)
fmodel = foolbox.TensorFlowModel(model, bounds=bounds, preprocessing=preprocessing)

2022-06-28 11:43:29.345599: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 11:43:29.345985: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 11:43:29.346171: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 11:43:29.346392: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-28 11:43:29.346557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [14]:
attack = foolbox.attacks.LinfAdamProjectedGradientDescentAttack()
for epsilon in [0.00125, 0.0025, 0.005, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32]:
    adv_batches = []
    labels = []
    print("epsilon =", epsilon)
    for i,(x,y) in tqdm(zip(range(len(test_batches)),test_batches), total=len(test_batches)):
        raw, clipped, is_adv = attack(fmodel, tf.constant(x), tf.argmax(y, axis=1), epsilons=[epsilon])
        adv_batches.append(clipped[0])
        labels.append(tf.argmax(y, axis=1))

    batch_accs = []
    batch_weights = []
    for (batch,label) in zip(adv_batches, labels):
        x = np.array(batch)
        y = np.array(label)
        acc = foolbox.utils.accuracy(fmodel, x, y)
        batch_accs.append(acc)
        batch_weights.append(y.shape[0])
    s1 = 0
    s2 = sum(batch_weights)
    for i in range(len(batch_accs)):
        s1 += batch_accs[i] * batch_weights[i]
    print("perturbed acc:",s1/s2)


epsilon = 0.00125


  0%|          | 0/440 [00:00<?, ?it/s]

perturbed acc: 0.8159430604982206
epsilon = 0.0025


  0%|          | 0/440 [00:00<?, ?it/s]

perturbed acc: 0.8152313167259786
epsilon = 0.005


...

perturbed acc: 0.7330960854092526
epsilon = 0.16


  0%|          | 0/440 [00:00<?, ?it/s]

perturbed acc: 0.5994306049822065
epsilon = 0.32


  0%|          | 0/440 [00:00<?, ?it/s]

perturbed acc: 0.28740213523131675


Using `FGSM()`
| Model/Epsilon     | 0                   | 0.00125             | 0.0025              | 0.005               | 0.01                | 0.02                | 0.04                |
| ----------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- |
| Without Attention | 0.832170818505338   | 0.8270462633451957  | 0.8241992882562278  | 0.8166548042704627  | 0.8024199288256227  | 0.7695373665480427  | 0.6908185053380783  |
| With Attention    | 0.817508896797153   | 0.8158007117437722  | 0.8146619217081851  | 0.8125266903914591  | 0.8075444839857652  | 0.7972953736654804  | 0.776085409252669   |

Using `LinfAdamProjectedGradientDescentAttack()`
| Model/Epsilon     | 0                   | 0.00125             | 0.0025              | 0.005               | 0.01                | 0.02                | 0.04                | 0.08                | 0.16                | 0.32                |
| ----------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- | ------------------- |
| Without Attention | 0.832170818505338   | 0.8277580071174377  | 0.8251957295373665  | 0.8186476868327403  | 0.805693950177936   | 0.7779359430604982  | 0.7051957295373665  | 0.5056227758007118  | 0.17480427046263344 | 0.01395017793594306 |
| With Attention    | 0.817508896797153   | 0.8159430604982206  | 0.8152313167259786  | 0.8132384341637011  | 0.8083985765124555  | 0.7998576512455516  | 0.7802135231316726 | 0.7330960854092526 | 0.5994306049822065 | 0.28740213523131675 |
